# 4일차 1교시 스파크 기본 명령어 이해

### 목차
* 1. Create SparkSession & DataFrames
* 2. What is Untyped Data ?
* 3. What is Structued API ?
* 4. Differences between SparkSQL, Dataframe and Dataset
* 5. References
  * [Spark Programming Guide](https://spark.apache.org/docs/latest/sql-programming-guide.html) 공식 문서를 참고하세요
  * [Spark on Github](https://github.com/apache/spark.git) 코드를 참고하세요
  * [Spark Definitive Guide](https://github.com/databricks/Spark-The-Definitive-Guide) 책에서 사용하는 데이터 예제를 활용했습니다
  * https://www.linkedin.com/pulse/apache-spark-rdd-vs-dataframe-dataset-chandan-prakash/
* 6. 공식 표준 문서 - 스파크와 파이썬 구문이 혼용되어 사용되므로 주의가 필요합니다.
  * <a href="https://spark.apache.org/docs/2.4.5/api/python/pyspark.sql.html" target="_blank">PySpark 2.4.5 Documentation - psypark.sql</a>
  * <a href="https://spark.apache.org/docs/2.4.5/api/sql/" target="_blank">PySpark 2.4.5 Builtin Functions</a>
  * <a href="https://docs.python.org/3.7/library/operator.html" target="_blank">Python 3.7.8 Documentation - operator</a>
  * <a href="https://docs.python.org/3/library/functools.html" target="_blank">Python 3.7.8 Documentation - functools</a>

### 1. Create SparkSession & DataFrames
> Spark 최신 버전인 2.3.1 버전을 기준으로 작성되었습니다. 스파크는 2.0 버전으로 업데이트 되면서 DataFrames 은 Datasets 으로 통합되었고, 기존의 RDD 에서 사용하던 연산 및 기능과 DataFrame 에서 사용하던 것 모두 사용할 수 있습니다. 스파크 데이터 모델은 RDD (Spark1.0) —> Dataframe(Spark1.3) —> Dataset(Spark1.6) 형태로 업그레이드 되었으나, 본문에서 일부 DataFrames 와 DataSets 가 거의 유사하여, 일부 혼용되어 사용되는 경우가 있을 수 있습니다.

In [4]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Data Engineer Intermediate Day4") \
    .config("spark.dataengineer.intermediate.day4", "tutorial-1") \
    .getOrCreate()

In [8]:
sc = spark.sparkContext
print("Spark " + sc.version)

# !which python
!/opt/conda/bin/python --version

Spark 2.4.5
Python 3.7.8


In [26]:
# 로컬 환경에서 파이썬 구문을 이용하여 1 ~ 100까지 더하는 함수를 구현합니다
result = 0
for number in range(1, 101, 1): result += number
print(result)

# 파이썬 3.0 에서는 reduce 함수를 사용할 수 있습니다
from functools import reduce 
reduce(lambda x, y: x + y, range(101))

5050


5050

In [27]:
# 분산환경에서는 스파크 Structured API 를 통해서 1 ~ 100 까지 더하는 함수를 구합니다.
from operator import add  # 파이썬의 operator 의 add 함수를 그대로 사용합니다.
parallels = sc.parallelize((range(1, 101, 1))).reduce(add)  # 1 ~ 101 이전까지 1씩 증가하는 숫자를 분산객체인 RDD를 반드시 생성해야 여러 노드의 메모리에 객체가 생성됩니다.
print(parallels)

x = sc.parallelize((range(1, 101, 1))).reduce(lambda x,y: x+y)  # 파이썬 람다함수를 이용해서 익명함수를 직접 생성해서 전달해도 결과는 동일합니다
print(x)

5050
5050


#### 로컬 환경에서 생성하는 객체와 분산환경의 객체는 생성하는 방식이 다른점을 주의할 필요가 있습니다
___

<br>
<br>

### 2. What is Untyped Data ?
#### Untyped Dataset Operations (aka Dataframe operations)
타입이 없다고 하면 잘 이해가 가지 않는데 자세한 설명은 추후에 하기로 하고 Java/Scala 와 같은 strong type 언어와는 다르게 type 에 대한 강한 제약 없이 기본적인 데이터 연산자들을 사용할 수 있다 정도로 이해하면 됩니다.

In [29]:
# activity-data : 다양한 장치 (특히 가속도계 및 자이로 스코프)의 스마트 폰 및 스마트 워치 센서 판독 값으로 구성된 다양한 사람의 활동 데이터 집합입니다.
df = spark.read.option("inferSchema", "true").json("data/activity-data/part-00079-tid-730451297822678341-1dda7027-2071-4d73-a0e2-7fb6a91e1d1f-0-c000.json")
df.printSchema()

root
 |-- Arrival_Time: long (nullable = true)
 |-- Creation_Time: long (nullable = true)
 |-- Device: string (nullable = true)
 |-- Index: long (nullable = true)
 |-- Model: string (nullable = true)
 |-- User: string (nullable = true)
 |-- gt: string (nullable = true)
 |-- x: double (nullable = true)
 |-- y: double (nullable = true)
 |-- z: double (nullable = true)



In [61]:
df.show(5)

+-------------+-------------------+--------+-----+------+----+-----+-------------+-------------+------------+
| Arrival_Time|      Creation_Time|  Device|Index| Model|User|   gt|            x|            y|           z|
+-------------+-------------------+--------+-----+------+----+-----+-------------+-------------+------------+
|1424686735175|1424688581230073365|nexus4_2|   43|nexus4|   g|stand|-0.0025177002| -0.054229736| 0.025863647|
|1424686735377|1424686733377625498|nexus4_1|   75|nexus4|   g|stand|-0.0039367676|   0.02507019| -0.01133728|
|1424686735577|1424688581632874879|nexus4_2|  123|nexus4|   g|stand| 0.0017547607|-0.0093688965|0.0012969971|
|1424686735776|1424686733780457529|nexus4_1|  155|nexus4|   g|stand| 0.0014038086|  0.014389038|-0.013473511|
|1424686735979|1424686733981873545|nexus4_1|  195|nexus4|   g|stand|-0.0018005371|  0.004776001| 0.023910522|
+-------------+-------------------+--------+-----+------+----+-----+-------------+-------------+------------+
only showi

#### 출력 시에 bigint 값인 날짜는 아래와 같이 from_unixtime 및 to_timestamp 함수를 통해 변환할 수 있습니다.

In [60]:
from pyspark.sql.functions import unix_timestamp, from_unixtime, to_timestamp, to_date
timestamp = df.select(
    "Arrival_Time",
    to_timestamp(from_unixtime(col('Arrival_Time') / lit(1000)), 'yyyy-MM-dd HH:mm:ss').alias('String_Datetime'),
    to_date(from_unixtime(col('Arrival_Time') / lit(1000)), 'yyyy-MM-dd').alias('String_Date')
)
timestamp.show(5)

+-------------+-------------------+-----------+
| Arrival_Time|    String_Datetime|String_Date|
+-------------+-------------------+-----------+
|1424686735175|2015-02-23 10:18:55| 2015-02-23|
|1424686735377|2015-02-23 10:18:55| 2015-02-23|
|1424686735577|2015-02-23 10:18:55| 2015-02-23|
|1424686735776|2015-02-23 10:18:55| 2015-02-23|
|1424686735979|2015-02-23 10:18:55| 2015-02-23|
+-------------+-------------------+-----------+
only showing top 5 rows



### 3. What is Structued API
#### Selecting Dataframe using structured APIs
> Since 2.4, when ***spark.sql.caseSensitive is set to false***, Spark does case insensitive column name resolution between Hive metastore schema and Parquet schema, so even column names are in different letter cases, Spark returns corresponding column values. An exception is thrown if there is ambiguity, i.e. more than one Parquet column is matched. This change also applies to Parquet Hive tables when spark.sql.hive.convertMetastoreParquet is set to true.

In [62]:
from pyspark.sql.functions import col
# 아래의 select 구문에서는 col("컬럼명") 혹은 "컬럼명" 둘다 혼용이 가능합니다.
df.filter(col("Index") > 100).select(col("Arrival_time"), col("Creation_Time"), col("Device")).groupBy("Device").count().show()

+--------+-----+
|  Device|count|
+--------+-----+
|nexus4_1|38637|
|nexus4_2|39351|
+--------+-----+



In [84]:
df.filter(col("Index") > 100).select(concat("Arrival_time", "Creation_Time"), "Device").show(5, truncate=False)

+-----------------------------------+--------+
|concat(Arrival_time, Creation_Time)|Device  |
+-----------------------------------+--------+
|14246867355771424688581632874879   |nexus4_2|
|14246867357761424686733780457529   |nexus4_1|
|14246867359791424686733981873545   |nexus4_1|
|14246867361811424686734183442148   |nexus4_1|
|14246867363831424686734387513193   |nexus4_1|
+-----------------------------------+--------+
only showing top 5 rows



In [89]:
# 아래와 같이 structured api 를 통해서 복잡한 구문을 selectExpr 을 통해 좀 더 편하게 조회할 수 있습니다.
df.filter(col("Index") > 100).selectExpr("concat('Arrival_time', 'Creation_Time') as Concated_Time", "Device").show(5, truncate=False)

+-------------------------+--------+
|Concated_Time            |Device  |
+-------------------------+--------+
|Arrival_timeCreation_Time|nexus4_2|
|Arrival_timeCreation_Time|nexus4_1|
|Arrival_timeCreation_Time|nexus4_1|
|Arrival_timeCreation_Time|nexus4_1|
|Arrival_timeCreation_Time|nexus4_1|
+-------------------------+--------+
only showing top 5 rows



In [6]:
df.filter(col("index") > 100).select("index", "user").groupBy("user").count().show()

+----+-----+
|user|count|
+----+-----+
|   g| 9165|
|   f| 9203|
|   e| 9599|
|   h| 7730|
|   d| 8122|
|   c| 7713|
|   i| 9253|
|   b| 9121|
|   a| 8082|
+----+-----+



In [91]:
# 대부분의 구문에서 표현식을 통해 처리할 수 있도록 내부적으로 2가지 방식에 대해 모두 구현되어 있습니다. 
df.filter("index > 100").select("index", "user").groupBy("user").count().show()

+----+-----+
|user|count|
+----+-----+
|   g| 9165|
|   f| 9203|
|   e| 9599|
|   h| 7730|
|   d| 8122|
|   c| 7713|
|   i| 9253|
|   b| 9121|
|   a| 8082|
+----+-----+



### 4. Differences between Dataframe, SparkSQL and Dataset

#### 4-1. Creating Dataframe from JSON file
> 임의의 JSON 파일로 부터 데이터프레임을 생성하고 집계를 수행할 수 있습니다.

In [93]:
json = spark.read.json("data/activity-data/part-00079-tid-730451297822678341-1dda7027-2071-4d73-a0e2-7fb6a91e1d1f-0-c000.json")
users = json.filter("index > 100").select("index", "user").groupBy("user").count()
users.show(5)

+----+-----+
|user|count|
+----+-----+
|   g| 9165|
|   f| 9203|
|   e| 9599|
|   h| 7730|
|   d| 8122|
+----+-----+
only showing top 5 rows



#### 4-2. Creating Temporary View
> 이미 생성된 데이터 프레임을 통해서 현재 세션에서만 조회 가능한 임시 뷰 테이블을 만들어 SQL 질의가 가능합니다.

In [95]:
users.createOrReplaceTempView("users")
spark.sql("select * from users where count is not null and count > 9000 order by count desc").show(5)

+----+-----+
|user|count|
+----+-----+
|   e| 9599|
|   i| 9253|
|   f| 9203|
|   g| 9165|
|   b| 9121|
+----+-----+



#### 4-3. Creating Global Temporary View
> 물론 글로벌한 뷰 테이블 생성도 가능하며, global_temp 데이터베이스에 생성되어 $SELECT * FROM\ global\_temp.people$ 과 같은 형식으로 조회가 가능합니다.

In [96]:
spark.catalog.dropGlobalTempView("global_users")
users.createGlobalTempView("global_users")
spark.sql("select * from global_temp.global_users").show(5)

+----+-----+
|user|count|
+----+-----+
|   g| 9165|
|   f| 9203|
|   e| 9599|
|   h| 7730|
|   d| 8122|
+----+-----+
only showing top 5 rows



In [97]:
newSession = """
public SparkSession newSession()
Start a new session with isolated SQL configurations, temporary tables, registered functions are isolated, 
but sharing the underlying SparkContext and cached data.
"""
spark.newSession().sql("select * from global_temp.global_users").show(5)

+----+-----+
|user|count|
+----+-----+
|   g| 9165|
|   f| 9203|
|   e| 9599|
|   h| 7730|
|   d| 8122|
+----+-----+
only showing top 5 rows

